# 4. Сверточные нейронные сети.

__Автор__: Никита Владимирович Блохин (NVBlokhin@fa.ru)

Финансовый университет, 2020 г.

In [16]:
import typing as t
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import metrics
from torch.utils.data import DataLoader, Dataset
from torchvision import models, datasets, transforms
from tqdm import tqdm

In [17]:
# @formatter:off
%matplotlib inline
# @formatter:on

In [18]:
torch.set_warn_always(True)

sns.set_theme()
plt.rcParams["figure.figsize"] = (8, 4)

In [19]:
DATA_DIR = Path("data/")
DATA_DIR.mkdir(exist_ok=True)
MODELS_DIR = Path("models/")
MODELS_DIR.mkdir(exist_ok=True)
COLAB_DATA_DIR = Path("/content/drive/MyDrive/Colab Notebooks/ML")

In [20]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {DEVICE} device")

Using cpu device


In [21]:
def is_colab() -> bool:
    return DEVICE == "cuda"


def train_loop(
        dataloader: DataLoader,
        model: nn.Module,
        loss_fn: nn.Module,
        optimizer: optim.Optimizer,
        verbose: int = 100,
) -> torch.Tensor:
    model.train()

    size = len(dataloader.dataset)  # noqa
    num_batches = len(dataloader)
    avg_loss = 0

    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(DEVICE), y.to(DEVICE)

        pred = model(x)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_loss += loss
        if batch % verbose == 0:
            print(f"loss: {loss:>7f}  [{batch * len(x):>5d}/{size:>5d}]")

    return avg_loss / num_batches


def test_loop(
        dataloader: DataLoader,
        model: nn.Module,
        loss_fn: nn.Module,
) -> t.Tuple[torch.Tensor, torch.Tensor]:
    model.eval()

    size = len(dataloader.dataset)  # noqa
    num_batches = len(dataloader)
    avg_loss, correct = 0, 0

    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            pred = model(x)
            avg_loss += loss_fn(pred, y)
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()  # noqa

    avg_loss /= num_batches
    accuracy = correct / size
    print(f"Test Error: \n Accuracy: {accuracy:>4f}, Avg loss: {avg_loss:>8f} \n")

    return avg_loss, accuracy


class FastDataset(Dataset):

    def __init__(self, dataset):
        self.dataset = dataset

        n = len(self.dataset)
        x, _ = self.dataset[0]
        self._data = torch.empty(n, *x.size(), dtype=x.dtype)
        self._targets = [0] * n
        for i, (x, y) in tqdm(enumerate(self.dataset), total=n):
            self._data[i] = x
            self._targets[i] = y

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self._data[idx], self._targets[idx]


def get_y_test_y_pred_batches(test_dataloader: DataLoader, model: nn.Module):
    model.eval()

    y_test = []
    y_pred = []
    for x, y in test_dataloader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        pred = model(x).argmax(1)
        y_test.append(y)
        y_pred.append(pred)

    return torch.hstack(y_test).detach().cpu(), torch.hstack(y_pred).detach().cpu()

# 4. Transfer Learning

### 4.1 Решить задачу 3.1, воспользовавшись предобученной моделью VGG16
* Загрузить данные для обучения
* Преобразования: размер 224x224, нормализация с параметрами `mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)`
* Заменить последний полносвязный слой модели в соответствии с задачей
* Дообучить модель (не замораживать веса). Вычислить значение accuracy на тестовом множестве
* Дообучить модель (заморозить все веса, кроме последнего блока слоев (`classifier`)). 
* Вычислить значение accuracy на тестовом множестве.


In [22]:
if is_colab():
    from google.colab import drive

    drive.mount('/content/drive')
    !cp -r "/content/drive/MyDrive/Colab Notebooks/ML/monkeys" "/content/data/"

In [23]:
transform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

train_monkeys_dataset = datasets.ImageFolder(
    DATA_DIR / "monkeys/training/training",
    transform=transform,
)
test_monkeys_dataset = datasets.ImageFolder(
    DATA_DIR / "monkeys/validation/validation",
    transform=transform,
)

(
    (len(train_monkeys_dataset), len(test_monkeys_dataset)),
    train_monkeys_dataset[0][0].shape,
    len(train_monkeys_dataset.classes),
)

((1097, 272), torch.Size([3, 224, 224]), 10)

In [24]:
class MonkeysVGG16(nn.Module):

    def __init__(self):
        super(MonkeysVGG16, self).__init__()
        self.vgg16 = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
        self.vgg16.classifier[-1] = nn.Linear(self.vgg16.classifier[-1].in_features, 10)

    def forward(self, x):
        return self.vgg16(x)

In [25]:
fast_train_monkeys_dataset = FastDataset(train_monkeys_dataset)
fast_test_monkeys_dataset = FastDataset(test_monkeys_dataset)

100%|██████████| 272/272 [00:08<00:00, 33.64it/s]


In [26]:
torch.manual_seed(0)

net = MonkeysVGG16().to(DEVICE)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

monkeys_train_dataloader = DataLoader(
    fast_train_monkeys_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=2 if is_colab() else 0,
)
monkeys_test_dataloader = DataLoader(
    fast_test_monkeys_dataset,
    batch_size=len(test_monkeys_dataset),
    num_workers=2 if is_colab() else 0,
)

net

MonkeysVGG16(
  (vgg16): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (

In [27]:
%%time
# дообучение всей модели

epochs = 10
train_losses = []
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}\n" + "-" * 32)
    train_loop(monkeys_train_dataloader, net, loss_fn, optimizer, verbose=50)
    test_loop(monkeys_test_dataloader, net, loss_fn)

Epoch 1
--------------------------------
loss: 20.000797  [    0/ 1097]
loss: 2.419504  [  400/ 1097]
loss: 2.379903  [  800/ 1097]


KeyboardInterrupt: 

In [29]:
%%time
# дообучение классификатора

for param in net.vgg16.features.parameters():
    param.requires_grad = False

optimizer = optim.Adam(net.vgg16.classifier.parameters(), lr=0.001)

epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}\n" + "-" * 32)
    train_loop(monkeys_train_dataloader, net, loss_fn, optimizer, verbose=50)
    test_loop(monkeys_test_dataloader, net, loss_fn)

Epoch 1
--------------------------------
loss: 2.473193  [    0/ 1097]


KeyboardInterrupt: 

In [30]:
y_test, y_pred = get_y_test_y_pred_batches(monkeys_test_dataloader, net)
print(metrics.classification_report(y_test, y_pred, target_names=test_monkeys_dataset.classes))

KeyboardInterrupt: 

### 4.2 Решить задачу 3.2, воспользовавшись подходящей предобученной моделью
* Не использовать VGG16 (вместо нее можно взять resnet18 или другую)
* Загрузить данные для обучения
* Преобразования: размер 224x224, нормализация с параметрами `mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)`
* Заменить последний полносвязный слой модели в соответствии с задачей
* Дообучить модель. 
* Вычислить значение accuracy на тестовом множестве (добиться значения не меньше 97-98%)